In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
sales = pd.read_csv("../../Dataset/FINAL_LinkedCleanSalesWeatherWithEncoding.csv",index_col="date",parse_dates=True)

In [4]:
sales.head()

,station_nbr,item_nbr,units,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,...,smoke,widespread_dust,sandstorm,squall,freezing,shallow,partial,patches,blowing,vicinity
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,2,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,3,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,4,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,5,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
sales.groupby("item_nbr")["units"].sum().sort_values(ascending=False).head()

item_nbr
45    1005111
9      916615
5      846662
44     577193
16     226772
Name: units, dtype: int64

Item <b>45</b> will be used as the main tester.

In [6]:
def getRegStr(df):
    lst = df.columns.values
    regStr = lst[2] + " ~ " + lst[3]
    for column in lst[4:]:
        regStr+=" + " + column
    return regStr       

In [7]:
def calculateMSE(test, target, model):
    test = test.reset_index()
    target = target.reset_index() 

    target["units_p"] = pd.DataFrame(model.predict(test))

    target.date = pd.to_datetime(target.date)
    target.set_index("date",inplace=True)

    target.units_p.fillna(0,inplace=True)
    
    # Rounding units did not improve MSE
    # Best MSE achieved by discarding prediction fractions
    #target.units_p = target.units_p.apply(round)
    target.units_p = target.units_p.astype(np.int64)
    target.units_p = target.units_p.apply(lambda x: 0 if x < 0 else x)

    return mean_squared_error(target.units,target.units_p)

In [8]:
def runRegression_forward(df, test_df, target_df, cols):
    results = pd.DataFrame(columns=["model","rsquared_adj","MSE"])
    
    for feature in df.columns[3:]:
        if feature in cols:
            continue
        tarcols = np.append(cols, feature);
        target = df[tarcols]
        reg = ols(getRegStr(target),df).fit()
        MSE = calculateMSE(test_df,target_df,reg)
        results = results.append(pd.DataFrame({"model":reg,"rsquared_adj":reg.rsquared_adj,"MSE" : MSE}, index = [feature]))
        
    return results

In [9]:
def saveFile(model, filename):
    pickle.dump(model, open(filename, 'wb'))
       
def loadFile(filename):
    return pickle.load(open(filename, 'rb'))

In [10]:
%%time
items = sales.item_nbr.unique()
foldsModels = pd.DataFrame(columns=["item_nbr","model","rsquared_adj","MSE"])
foldsGenerator = KFold(n_splits=5,shuffle=True,random_state=0)

#for item in items:       # UN-comment to run on all items WARNING: runtime duration (~ 3-5 hrs).
for item in range(1):     # UN-comment to run on single item only.
    item = 45             # UN-comment to run on single item only.

    itemSales = sales.loc[sales.item_nbr == item].copy()
    
    ########################################################
    # TEST if any Feature is causing the regression to fail.    
    regAnalysis = pd.DataFrame(columns=["feature","rsquared_adj"])
    
    for feature in itemSales.columns[3:]:
        reg = ols("units~"+feature,itemSales).fit()
        regAnalysis = regAnalysis.append(pd.DataFrame({"feature":feature,"rsquared_adj" : reg.rsquared_adj}, index = [item]))
    badFeatures = regAnalysis.loc[regAnalysis.rsquared_adj.isnull()]
        
    itemSales.drop(badFeatures.feature.values,axis=1,inplace=True)
    if len(badFeatures)>0:
        print("Item [",item,"] Pre-processing: Bad Features dropped:",badFeatures.feature.values,"\n")
        print("Item [",item,"] Pre-processing: Remaining Features length",len(itemSales.columns)-3,"\n")
        if (len(itemSales.columns)-3) == 0:
            continue       # filter out items with no features to analyze.
    else:
        print("Item [",item,"] Pre-processing: Bad Features dropped: NONE\n")
    ########################################################
   
    folds = list(foldsGenerator.split(itemSales))

    for fold in range(len(folds)):   # Un-comment to run all Folds.
    #for fold in range(1):           # Un-comment to run Fold1 only.
        
        train, test = folds[fold]

        train_itemSales = itemSales.iloc[train]
        
        test_itemSales = itemSales.iloc[test]
        target_itemSales = itemSales.iloc[test][["units"]]
        
        orgModel = ols(getRegStr(train_itemSales),train_itemSales).fit()

        orgMSE = calculateMSE(test_itemSales,target_itemSales,orgModel)
        
        # forward selection base should be 0
        baseReg = 0
        baseMSE = orgMSE
        
        opt_itemSales = train_itemSales.copy()
        
        # initial columns for forward selection
        cols = ['station_nbr','item_nbr', 'units']

        print("#####","Item [",item,"] Fold",fold+1,"#####")
        for i in range(len(opt_itemSales.columns[3:])):
            
            regAnalysis = runRegression_forward(opt_itemSales, test_itemSales, target_itemSales, cols)

            maxReg = regAnalysis.loc[[regAnalysis.rsquared_adj.idxmax()]]
            minMSE = regAnalysis.loc[[regAnalysis.MSE.idxmin()]]
            
            ##############################################################
            #    Select Criteria HERE by setting only one mode as TRUE
            #-------------------------------------------------------------
            select_by_Radj = False
            select_by_MSE = True
            
            if select_by_Radj:
                criteria = maxReg.rsquared_adj[0] > baseReg 
            elif select_by_MSE:
                criteria =  baseMSE > minMSE.MSE[0]
            #####################################################

            # Comment out to reduce printout.
            print("-----------------------------------------------------------------")
            print("Round:", i+1)
                       
            if select_by_Radj:
                print("Current Base reg:",baseReg)
                print("         Max Reg:", maxReg.rsquared_adj[0],"["+maxReg.index[0]+"]")
            elif select_by_MSE:
                print("Current Base MSE:",baseMSE)
                print("         Min MSE:", minMSE.MSE[0],"["+minMSE.index[0]+"]")

            if criteria:
                if select_by_Radj: 
                    cols.append(maxReg.index[0])
                elif select_by_MSE:
                    cols.append(minMSE.index[0])
                    
                baseReg = maxReg.rsquared_adj[0]
                baseMSE = minMSE.MSE[0]
                # Comment out to reduce printout.
                print("Continue")
                
            else:
                # Comment out to reduce printout.
                print("Break")
                print("-----------------------------------------------------------------")
                print("Item [",item,"] Fold",fold+1,"Results:")
                
                if select_by_Radj:
                    print(" Orignal score:", orgModel.rsquared_adj)
                    print("Improved score:", maxReg.rsquared_adj[0])
                    rsquared_adj = maxReg.rsquared_adj[0]
                    mse = maxReg.MSE[0]
                    optModel = maxReg.model[0]
                elif select_by_MSE:
                    print(" Orignal MSE:", orgMSE)
                    print("Improved MSE:", baseMSE)
                    rsquared_adj = minMSE.rsquared_adj[0]
                    mse = minMSE.MSE[0]
                    optModel = minMSE.model[0]
                    
                print("=================================================================")
                break

        foldsModels = foldsModels.append(pd.DataFrame({"item_nbr":item,"model": optModel,\
                                                       "rsquared_adj" : rsquared_adj,\
                                                       "MSE" : mse},\
                                                      index = ["fold"+str(fold+1)]))
foldsModels.index.name = "Folds"
foldsModels = foldsModels[["item_nbr","model","rsquared_adj","MSE"]]     #rearrange columns

print("\n ***** Analysis Completed *****\n")

Item [ 45 ] Pre-processing: Bad Features dropped: NONE

##### Item [ 45 ] Fold 1 #####
-----------------------------------------------------------------
Round: 1
Current Base MSE: 13478.5228634
         Min MSE: 6854.81654872 [stnpressure]
Continue
-----------------------------------------------------------------
Round: 2
Current Base MSE: 6854.81654872
         Min MSE: 6703.96216658 [sealevel]
Continue
-----------------------------------------------------------------
Round: 3
Current Base MSE: 6703.96216658
         Min MSE: 6655.83451279 [vicinity]
Continue
-----------------------------------------------------------------
Round: 4
Current Base MSE: 6655.83451279
         Min MSE: 6615.04572673 [resultdir]
Continue
-----------------------------------------------------------------
Round: 5
Current Base MSE: 6615.04572673
         Min MSE: 6589.91834513 [resultspeed]
Continue
-----------------------------------------------------------------
Round: 6
Current Base MSE: 6589.91834513
    

-----------------------------------------------------------------
Round: 13
Current Base MSE: 6456.40539069
         Min MSE: 6455.74462292 [rain]
Continue
-----------------------------------------------------------------
Round: 14
Current Base MSE: 6455.74462292
         Min MSE: 6454.11489246 [shallow]
Continue
-----------------------------------------------------------------
Round: 15
Current Base MSE: 6454.11489246
         Min MSE: 6453.30520011 [fog]
Continue
-----------------------------------------------------------------
Round: 16
Current Base MSE: 6453.30520011
         Min MSE: 6452.04492241 [wetbulb]
Continue
-----------------------------------------------------------------
Round: 17
Current Base MSE: 6452.04492241
         Min MSE: 6451.2153553 [smoke]
Continue
-----------------------------------------------------------------
Round: 18
Current Base MSE: 6451.2153553
         Min MSE: 6450.66893548 [heat]
Continue
------------------------------------------------------------

-----------------------------------------------------------------
Round: 19
Current Base MSE: 7392.62673564
         Min MSE: 7391.96460659 [smoke]
Continue
-----------------------------------------------------------------
Round: 20
Current Base MSE: 7391.96460659
         Min MSE: 7391.01878573 [haze]
Continue
-----------------------------------------------------------------
Round: 21
Current Base MSE: 7391.01878573
         Min MSE: 7390.7010618 [drizzle]
Continue
-----------------------------------------------------------------
Round: 22
Current Base MSE: 7390.7010618
         Min MSE: 7390.44105636 [squall]
Continue
-----------------------------------------------------------------
Round: 23
Current Base MSE: 7390.44105636
         Min MSE: 7389.91423904 [preciptotal]
Continue
-----------------------------------------------------------------
Round: 24
Current Base MSE: 7389.91423904
         Min MSE: 7389.85189219 [small_hail]
Continue
-----------------------------------------------

In [22]:
# minimize result
foldsModels.set_index([foldsModels.index,"item_nbr"],inplace = True)

optFolds = foldsModels.groupby("item_nbr")["MSE"].idxmin().values

optFoldsModels = foldsModels.loc[optFolds]

In [ ]:
optFoldsModels

In [10]:
saveFile(foldsModels,"ForwardRegressionMSE")

In [5]:
foldsModels = loadFile('ForwardRegressionMSE')

In [8]:
foldsModels['selection'] = 'forward'

In [9]:
foldsModels

model  \
Folds item_nbr                                                      
fold5 1         <statsmodels.regression.linear_model.Regressio...   
fold1 2         <statsmodels.regression.linear_model.Regressio...   
fold2 3         <statsmodels.regression.linear_model.Regressio...   
fold1 4         <statsmodels.regression.linear_model.Regressio...   
      5         <statsmodels.regression.linear_model.Regressio...   
      6         <statsmodels.regression.linear_model.Regressio...   
fold5 7         <statsmodels.regression.linear_model.Regressio...   
fold1 8         <statsmodels.regression.linear_model.Regressio...   
fold3 9         <statsmodels.regression.linear_model.Regressio...   
      10        <statsmodels.regression.linear_model.Regressio...   
fold2 11        <statsmodels.regression.linear_model.Regressio...   
fold5 12        <statsmodels.regression.linear_model.Regressio...   
fold2 13        <statsmodels.regression.linear_model.Regressio...   
fold5 14        <statsmodels.regression.linear_model.Regressio...   
fold3 15        <statsmodels.regression.linear_model.Regressio...   
fold1 16        <statsmodels.regression.linear_model.Regressio...   
fold5 17        <statsmodels.regression.linear_model.Regressio...   
fold3 18        <statsmodels.regression.linear_model.Regressio...   
fold1 19        <statsmodels.regression.linear_model.Regressio...   
      20        <statsmodels.regression.linear_model.Regressio...   
fold3 21        <statsmodels.regression.linear_model.Regressio...   
fold5 22        <statsmodels.regression.linear_model.Regressio...   
fold3 23        <statsmodels.regression.linear_model.Regressio...   
fold1 24        <statsmodels.regression.linear_model.Regressio...   
fold2 25        <statsmodels.regression.linear_model.Regressio...   
fold4 26        <statsmodels.regression.linear_model.Regressio...   
fold1 27        <statsmodels.regression.linear_model.Regressio...   
fold4 28        <statsmodels.regression.linear_model.Regressio...   
fold1 29        <statsmodels.regression.linear_model.Regressio...   
      30        <statsmodels.regression.linear_model.Regressio...   
...                                                           ...   
fold2 82        <statsmodels.regression.linear_model.Regressio...   
fold4 83        <statsmodels.regression.linear_model.Regressio...   
fold3 84        <statsmodels.regression.linear_model.Regressio...   
fold4 85        <statsmodels.regression.linear_model.Regressio...   
fold2 86        <statsmodels.regression.linear_model.Regressio...   
fold5 87        <statsmodels.regression.linear_model.Regressio...   
fold1 88        <statsmodels.regression.linear_model.Regressio...   
fold4 89        <statsmodels.regression.linear_model.Regressio...   
      90        <statsmodels.regression.linear_model.Regressio...   
fold5 91        <statsmodels.regression.linear_model.Regressio...   
fold1 92        <statsmodels.regression.linear_model.Regressio...   
fold2 93        <statsmodels.regression.linear_model.Regressio...   
fold1 94        <statsmodels.regression.linear_model.Regressio...   
fold3 95        <statsmodels.regression.linear_model.Regressio...   
fold4 96        <statsmodels.regression.linear_model.Regressio...   
fold2 97        <statsmodels.regression.linear_model.Regressio...   
      98        <statsmodels.regression.linear_model.Regressio...   
fold1 99        <statsmodels.regression.linear_model.Regressio...   
fold5 100       <statsmodels.regression.linear_model.Regressio...   
      101       <statsmodels.regression.linear_model.Regressio...   
      102       <statsmodels.regression.linear_model.Regressio...   
fold2 103       <statsmodels.regression.linear_model.Regressio...   
fold1 104       <statsmodels.regression.linear_model.Regressio...   
      105       <statsmodels.regression.linear_model.Regressio...   
fold2 106       <statsmodels.regression.linear_model.Regressio...   
      107       <statsmodels.regression.linear_model.Regre